In [18]:
import requests
import pandas as pd
import sqlite3

In [ ]:
### Deze notebook is voor het ophalen van buienalarm data, opzetten van de datasets en het opslaan van de datasets
# link naar ERD voor stations/measurements: https://drive.google.com/file/d/1foX7bRjfRhr7RsuGN5_LE3gzuj3nfMyd/view?usp=sharing

In [19]:
# Use get request to fetch data from endpoint
endpoint = 'https://data.buienradar.nl/2.0/feed/json'

response = requests.get(url=endpoint)
response.raise_for_status()  # Raise an error if request failed

#note: only returns most recent measurements
data = response.json()
print(data)

#Transform to pandas dataframe
data = data['actual']['stationmeasurements']
data = pd.DataFrame(data)
data.head()

{'$id': '1', 'buienradar': {'$id': '2', 'copyright': '(C)opyright Buienradar / RTL. Alle rechten voorbehouden', 'terms': 'Deze feed mag vrij worden gebruikt onder voorwaarde van bronvermelding buienradar.nl inclusief een hyperlink naar https://www.buienradar.nl. Aan de feed kunnen door gebruikers of andere personen geen rechten worden ontleend.'}, 'actual': {'$id': '3', 'actualradarurl': 'https://api.buienradar.nl/image/1.0/RadarMapNL?w=500&h=512', 'sunrise': '2025-09-01T06:47:00', 'sunset': '2025-09-01T20:26:00', 'stationmeasurements': [{'$id': '4', 'stationid': 6391, 'stationname': 'Meetstation Arcen', 'lat': 51.5, 'lon': 6.2, 'regio': 'Venlo', 'timestamp': '2025-09-01T13:30:00', 'weatherdescription': 'Mix van opklaringen en middelbare of lage bewolking', 'iconurl': 'https://cdn.buienradar.nl/resources/images/icons/weather/30x30/b.png', 'fullIconUrl': 'https://cdn.buienradar.nl/resources/images/icons/weather/96x96/B.png', 'graphUrl': 'https://www.buienradar.nl/nederland/weerbericht/w

,$id,stationid,stationname,lat,lon,regio,timestamp,weatherdescription,iconurl,fullIconUrl,...,windspeed,windspeedBft,humidity,precipitation,sunpower,rainFallLast24Hour,rainFallLastHour,winddirectiondegrees,airpressure,visibility
0,4,6391,Meetstation Arcen,51.50,6.20,Venlo,2025-09-01T13:30:00,Mix van opklaringen en middelbare of lage bewo...,https://cdn.buienradar.nl/resources/images/ico...,https://cdn.buienradar.nl/resources/images/ico...,...,1.5,1.0,73.0,0.0,626.0,9.7,0.0,211.0,NaN,NaN
1,5,6275,Meetstation Arnhem,52.07,5.88,Arnhem,2025-09-01T13:30:00,Zwaar bewolkt,https://cdn.buienradar.nl/resources/images/ico...,https://cdn.buienradar.nl/resources/images/ico...,...,4.1,3.0,75.0,0.0,742.0,4.1,0.0,172.0,1007.4,25600.0
2,6,6249,Meetstation Berkhout,52.65,4.98,Berkhout,2025-09-01T13:30:00,Vrijwel onbewolkt (zonnig/helder),https://cdn.buienradar.nl/resources/images/ico...,https://cdn.buienradar.nl/resources/images/ico...,...,4.9,3.0,63.0,0.0,666.0,0.2,0.0,207.0,NaN,21100.0
3,7,6260,Meetstation De Bilt,52.10,5.18,Utrecht,2025-09-01T13:30:00,Zwaar bewolkt,https://cdn.buienradar.nl/resources/images/ico...,https://cdn.buienradar.nl/resources/images/ico...,...,4.0,3.0,69.0,0.0,518.0,0.0,0.0,186.0,1006.7,26700.0
4,8,6235,Meetstation Den Helder,52.92,4.78,Den Helder,2025-09-01T13:30:00,Zwaar bewolkt,https://cdn.buienradar.nl/resources/images/ico...,https://cdn.buienradar.nl/resources/images/ico...,...,8.8,5.0,66.0,0.0,858.0,0.0,0.0,217.0,1005.4,26000.0


In [20]:
# Create measurements dataset
measurements = data[['stationid','timestamp', 'temperature', 'groundtemperature', 'feeltemperature', 'windgusts', 'windspeedBft', 'humidity', 'precipitation', 'sunpower']].drop_duplicates()

# Index measurement by joining station index by measurement ts with '_'
measurements['measurementid'] = (
    measurements['stationid'].astype(str) + '_' + measurements['timestamp'].astype(str)
)
measurements.head(10)

,stationid,timestamp,temperature,groundtemperature,feeltemperature,windgusts,windspeedBft,humidity,precipitation,sunpower,measurementid
0,6391,2025-09-01T13:30:00,20.3,20.7,20.3,4.3,1.0,73.0,0.0,626.0,6391_2025-09-01T13:30:00
1,6275,2025-09-01T13:30:00,20.0,21.3,20.0,7.0,3.0,75.0,0.0,742.0,6275_2025-09-01T13:30:00
2,6249,2025-09-01T13:30:00,21.4,23.5,21.4,7.9,3.0,63.0,0.0,666.0,6249_2025-09-01T13:30:00
3,6260,2025-09-01T13:30:00,20.3,22.0,20.3,6.6,3.0,69.0,0.0,518.0,6260_2025-09-01T13:30:00
4,6235,2025-09-01T13:30:00,20.4,22.7,20.4,11.4,5.0,66.0,0.0,858.0,6235_2025-09-01T13:30:00
5,6370,2025-09-01T13:30:00,21.3,22.6,21.3,8.0,3.0,61.0,0.0,407.0,6370_2025-09-01T13:30:00
6,6350,2025-09-01T13:30:00,23.6,27.3,25.4,12.0,4.0,44.0,0.0,793.0,6350_2025-09-01T13:30:00
7,6323,2025-09-01T13:30:00,22.2,23.9,25.1,11.6,4.0,51.0,0.0,715.0,6323_2025-09-01T13:30:00
8,6283,2025-09-01T13:30:00,18.4,18.5,18.4,6.1,2.0,84.0,0.0,203.0,6283_2025-09-01T13:30:00
9,6280,2025-09-01T13:30:00,18.9,19.1,18.9,5.2,3.0,83.0,0.0,233.0,6280_2025-09-01T13:30:00


In [25]:
#Create stations dataset
stations = data[['stationid', 'stationname', 'lat', 'lon', 'regio']].drop_duplicates()
stations.head()

,stationid,stationname,lat,lon,regio
0,6391,Meetstation Arcen,51.50,6.20,Venlo
1,6275,Meetstation Arnhem,52.07,5.88,Arnhem
2,6249,Meetstation Berkhout,52.65,4.98,Berkhout
3,6260,Meetstation De Bilt,52.10,5.18,Utrecht
4,6235,Meetstation Den Helder,52.92,4.78,Den Helder


In [ ]:
# Store into a local database
db_path = "/Users/jinhan.bai/Software/personal/skilltest-zypp/skilltest-buienradar/database/weather.db"

conn = sqlite3.connect(db_path)

# Enforce foreign keys in SQLite
conn.execute("PRAGMA foreign_keys = ON")

with conn:
    # Stations table
    conn.execute("""
        CREATE TABLE IF NOT EXISTS stations (
            stationid TEXT PRIMARY KEY,
            stationname TEXT,
            lat REAL,
            lon REAL,
            regio REAL
        )
    """)

    # Measurements table
    conn.execute("""
        CREATE TABLE IF NOT EXISTS measurements (
            measurementid TEXT PRIMARY KEY,
            stationid TEXT NOT NULL,
            timestamp TEXT NOT NULL,
            temperature REAL,
            groundtemperature REAL,
            feeltemperature REAL,
            windgusts REAL,
            windspeedBft REAL,
            humidity REAL,
            precipitation REAL,
            sunpower REAL,
            FOREIGN KEY (stationid) REFERENCES stations(stationid) ON DELETE CASCADE
        )
    """)

    # Useful indexes and constraints
    conn.execute("CREATE UNIQUE INDEX IF NOT EXISTS uq_measurements_stationid_timestamp ON measurements(stationid, timestamp)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_measurements_stationid ON measurements(stationid)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_measurements_timestamp ON measurements(timestamp)")

# Define dtype mappings
stations_dtype = {
    'stationid': 'TEXT',
    'stationname': 'TEXT',
    'lat': 'REAL',
    'lon': 'REAL',
    'regio': 'TEXT'
}

measurements_dtype = {
    'measurementid': 'TEXT',
    'stationid': 'TEXT',
    'timestamp': 'TEXT',
    'temperature': 'REAL',
    'groundtemperature': 'REAL',
    'feeltemperature': 'REAL',
    'windgusts': 'REAL',
    'windspeedBft': 'INTEGER',
    'humidity': 'REAL',
    'precipitation': 'REAL',
    'sunpower': 'REAL'
}

# Write data (append to preserve schema, keys, and indexes)
stations.to_sql("stations", conn, if_exists="replace", index=False, dtype=stations_dtype)
measurements.to_sql("measurements", conn, if_exists="replace", index=False, dtype=measurements_dtype)

conn.close()